In [1]:
import recognitionTypes.hardCodedRecognition as hard_coded
from recognitionTypes.classificatorsRecognition import ClassificatorRecognition as classificator
from recognitionTypes.neuralNetworkRecognition import NeuralRecognition as neuralRecognition
import cv2 as cv2
import mediapipe as mp
import pandas as pd

from threading import *
import time
from queue import Queue

gestureNameList = ["forward", "backward", "left", "right", "up", "down", "rotate left", "rotate right", "stop", "land"]
gestureName = ""
gestureID = 0

event=Event()
def countdown(i):
    while i > 0:
        print(i)
        i-=1
        time.sleep(1)

In [2]:
neural = neuralRecognition()
neural.load_saved_model()

In [3]:
def gestureRecognThread(output_queue, landmark_queue, w, h, classifier):
    global gestureName
    while True:
        event.wait()
        while event.isSet():
            landmarks = landmark_queue.get()
            gestureName, gestureID = classifier.getGestureName(landmarks,w,h)
            # print(f"{gestureName}")
            output_queue.put(gestureID)

def flyDrone(input_queue, img):
    global gestureName
    while True:
        event.set()
        gestureID = input_queue.get()
        print(f"Drone will do the following: {gestureName} with the id {gestureID}")
        time.sleep(2)
        event.clear()


In [4]:
def mainprogram(recognitionType):
    cap = cv2.VideoCapture(0)

    mpHands = mp.solutions.hands
    hands = mpHands.Hands( max_num_hands=1, min_detection_confidence=0.9, static_image_mode=False)
    mpDraw = mp.solutions.drawing_utils
    isSet = 0
    landmarkQueue = Queue()
    communicationQueue = Queue()
    while True:
        success, img = cap.read()
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img,1)
        results = hands.process(img)
        h, w, c = img.shape
        cv2.rectangle(img, (20,20), (300,400), (255,0,0), 2)

        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                # for id, lm in enumerate(handLms.landmark):
                #     cx, cy = int(lm.x * w), int(lm.y * h)

                landMarkVect = handLms.landmark
                hard_coded.checkPointsRange(landMarkVect, w, h)
                landmarkQueue.put(landMarkVect)
                if(isSet == 0):
                    t1 = Thread(target=gestureRecognThread, args=(communicationQueue,landmarkQueue,w,h,recognitionType))
                    t2 = Thread(target=flyDrone, args=(communicationQueue,img))
                    t2.start()
                    t1.start()
                    isSet=1
                
                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
                cv2.putText(img, gestureName, (10, 70), cv2.FONT_ITALIC, 1, (0, 0, 255), 1)

        cv2.imshow("Image", img)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            cv2.destroyAllWindows()
            break

In [5]:
mainprogram(neural)

Drone will do the following: stop with the id 8
Drone will do the following: stop with the id 8
Drone will do the following: forward with the id 8
Drone will do the following: forward with the id 8
Drone will do the following: forward with the id 8
Drone will do the following: rotate right with the id 8
Drone will do the following: up with the id 8
Drone will do the following: rotate right with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the id 8
Drone will do the following: up with the 